In [1]:
import numpy as np
import pandas as pd
import os
import re

In [2]:
jobs = pd.read_csv("DataSet/JobPostDetails/data.csv")
numbeo = {}
numbeo_set = {}
for dataset in os.listdir("DataSet/Numbeo Data"):
    data = dataset.split(".")[0]
    numbeo[data] = pd.read_csv("DataSet/Numbeo Data/" + dataset)
    numbeo[data].drop("Rank", axis = 1, inplace=True)
    numbeo_set[data] = set(numbeo[dataset.split(".")[0]]['City'].unique())

In [3]:
reverse_map = {'Abbotsford': 'Abbotsford, BC',
         'Barrie': 'Barrie, ON',
         'Brampton': "Brampton, ON",
         'Burlington': 'Burlington, ON',
         'Burnaby': 'Burnaby, BC',
         'Calgary': 'Calgary, AB',
         'Coquitlam': 'Coquitlam, BC',
         'Edmonton': 'Edmonton, AB',
         'Fredericton': 'Fredericton, NB',
         'Guelph': 'Guelph, ON',
         'Halifax': 'Halifax, NS',
         'Hamilton': 'Hamilton, ON',
         'Kamloops': 'Kamloops, BC',
         'Kelowna': 'Kelowna, BC',
         'Kingston': 'Kingston, ON',
         'Kitchener': 'Kitchener, ON',
         'Langley, BC': 'Langley, BC',
         'Lethbridge, AB': 'Lethbridge, AB',
         'London':'London, ON',
         'Markham': 'Markham ON',
         'Mississauga': 'Mississauga, ON',
         'Moncton': 'Moncton, NB',
         'Montreal': 'Montreal, QC',
         'Nanaimo, BC': 'Nanaimo, BC',
         'New Westminster': 'New Westminster, BC',
         'North Vancouver': 'North Vancouver, BC',
         'Oakville': 'Oakville, ON',
         'Oshawa': 'Oshawa, ON',
         'Ottawa': 'Ottawa, ON',
         'Quebec City': 'Quebec City, QC',
         'Red Deer': 'Red Deer, AB',
         'Regina': 'Regina, SK',
         'Richmond Hill': 'Richmond Hill, ON',
         "Saint John's": "Saint John's, NL",
         'Saskatoon': 'Saskatoon, SK',
         'St.Catharines': 'St.Catharines, ON',
         'Sudbury': 'Sudbury, ON',
         'Surrey': 'Surrey, BC',
         'Thunder Bay': 'Thunder Bay, ON',
         'Toronto': 'Toronto, ON',
         'Vancouver':'Vancouver, BC',
         'Vaughan': 'Vaughan, ON',
         'Victoria': 'Victoria, BC',
         'Waterloo': 'Waterloo, ON',
         'Windsor': 'Windsor, ON',
         'Winnipeg': 'Winnipeg, MB'
              }

In [4]:
city_set = set()
for key, value in numbeo_set.items():
    city_set = city_set.union(value)

In [5]:
regex = "[A-Za-z]\d[A-Za-z][ -]?\d[A-Za-z]"

In [7]:
addresses = [city for city  in jobs["City"].unique() if re.search(regex, city)]
non_addresses = [city for city  in jobs["City"].unique() if not re.search(regex, city)]

In [8]:
reverse_address = {address: reverse_map[city] for address in addresses for city in city_set if city in address }

In [9]:
relevent_jobs = jobs.copy(deep=True)

In [10]:
def filter_address(x):
    if x in reverse_address.keys():
        return reverse_address[x]
    else:
        return x
relevent_jobs["City"] = relevent_jobs["City"].apply(filter_address)

In [11]:
relevent_jobs = jobs[jobs["City"].isin(non_addresses) | jobs["City"].isin(city_set)]
relevent_jobs["City_Split"] = relevent_jobs["City"].apply(lambda x: x.split(",")[0])

<ipython-input-11-b77ae6434915>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevent_jobs["City_Split"] = relevent_jobs["City"].apply(lambda x: x.split(",")[0])


In [12]:
numbeo_join = relevent_jobs[relevent_jobs["City"].isin([reverse_map[city] for city in city_set])]
for key, group in numbeo_join.groupby("City"):
    print(key, len(group))

Abbotsford, BC 527
Barrie, ON 87
Brampton, ON 533
Burlington, ON 114
Burnaby, BC 463
Calgary, AB 603
Coquitlam, BC 179
Edmonton, AB 709
Fredericton, NB 89
Guelph, ON 105
Halifax, NS 207
Hamilton, ON 163
Kamloops, BC 124
Kelowna, BC 208
Kingston, ON 52
Kitchener, ON 93
Langley, BC 254
Lethbridge, AB 51
London, ON 128
Mississauga, ON 709
Moncton, NB 157
Nanaimo, BC 111
New Westminster, BC 109
North Vancouver, BC 192
Oakville, ON 146
Oshawa, ON 46
Ottawa, ON 317
Red Deer, AB 42
Regina, SK 451
Richmond Hill, ON 135
Saskatoon, SK 404
Sudbury, ON 113
Surrey, BC 2503
Thunder Bay, ON 59
Toronto, ON 722
Vancouver, BC 1079
Vaughan, ON 17
Victoria, BC 285
Waterloo, ON 73
Windsor, ON 72
Winnipeg, MB 395


In [13]:
joined = {}
for key, value in numbeo.items():
    value["City"] = value["City"].apply(lambda x: reverse_map[x])
    joined[key] = pd.merge(numbeo_join, value, on="City", how="inner")

In [23]:
merged = pd.merge(numbeo["PropertyPrices"], numbeo["CostofLiving"], how="inner", on="City")
q = pd.merge(numbeo_join, merged, on="City", how="inner")
q[q["PayUnit"] == "HOUR"].corr()

,Price To Income Ratio,Gross Rental Yield City Centre,Gross Rental Yield Outside of Centre,Price To Rent Ratio City Centre,Price To Rent Ratio Outside Of City Centre,Mortgage As A Percentage Of Income,Affordability Index,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
Price To Income Ratio,1.000000,-0.829168,-0.773598,0.883664,0.783112,0.994350,-0.913225,0.323746,0.908107,0.876264,0.324356,0.115457,-0.470112
Gross Rental Yield City Centre,-0.829168,1.000000,0.737697,-0.957215,-0.676718,-0.827451,0.822420,-0.445759,-0.818708,-0.834514,-0.481004,-0.059985,0.161301
Gross Rental Yield Outside of Centre,-0.773598,0.737697,1.000000,-0.764732,-0.951767,-0.762160,0.767631,-0.381896,-0.692566,-0.707291,-0.442799,-0.103023,0.084499
Price To Rent Ratio City Centre,0.883664,-0.957215,-0.764732,1.000000,0.779007,0.866367,-0.801969,0.485066,0.834397,0.859540,0.480939,0.159841,-0.156068
Price To Rent Ratio Outside Of City Centre,0.783112,-0.676718,-0.951767,0.779007,1.000000,0.754362,-0.691956,0.376925,0.691712,0.705181,0.393187,0.245691,-0.023399
Mortgage As A Percentage Of Income,0.994350,-0.827451,-0.762160,0.866367,0.754362,1.000000,-0.940205,0.321207,0.904175,0.872114,0.346577,0.062448,-0.522586
Affordability Index,-0.913225,0.822420,0.767631,-0.801969,-0.691956,-0.940205,1.000000,-0.258439,-0.847105,-0.804617,-0.393877,0.141451,0.570181
Cost of Living Index,0.323746,-0.445759,-0.381896,0.485066,0.376925,0.321207,-0.258439,1.000000,0.347781,0.589636,0.860574,0.357044,0.069165
Rent Index,0.908107,-0.818708,-0.692566,0.834397,0.691712,0.904175,-0.847105,0.347781,1.000000,0.962315,0.347512,0.074098,-0.257407
Cost of Living Plus Rent Index,0.876264,-0.834514,-0.707291,0.859540,0.705181,0.872114,-0.804617,0.589636,0.962315,1.000000,0.548922,0.167384,-0.201769


In [15]:
numbeo.keys()

dict_keys(['Crime', 'Traffic', 'HealthCare', 'Quality of Life', 'CostofLiving', 'PropertyPrices'])